# Jacobi and Gauss Seidel Method 

In this Part we are going to see 2 other iterative methods to solve linear systems,Jacobi Method and Gauss seidel Method

## 1-Jacobi Method

Jacobi method is an iterative method to solve systems $Ax = b$ by rewriting the system (under conditions on A) in an equivalent one $x = Bx + f$ and consider the sequence $x^{n+1} = Bx^{n} + f$ where  $\lim_{n\to\infty}  \lVert x - x^n \rVert = 0 $, by starting from a point in space we endup (under conditions on A) to converge
to $x$ the solution of $Ax = b$

Let rewrite $A = D - E - F$ where 

* D is the diagonal elements of $A$

* $E$ is - the lower triangular matrix of $A$ where $e_{ij} = -a_{ij}$ for  $i > j$ else $e_{ij} = 0$

* F is - the upper triangular matrix of $A$ where $e_{ij} = -a_{ij}$ for  $i < j$ else $e_{ij} = 0$

We obtain  
$
\begin{align}
Ax = b \iff (D - E - F) x = b \\
\iff Dx =(E + F)x + b \\
\iff x = D^{-1}(E + F)x + D^{-1}b\\
\iff x = Bx + f \; \\where \; B =  D^{-1}(E + F)= I - D^{-1} A \; and \; f =  D^{-1}b
\end{align}
$

### Condition of existence :

Let define the error $e^{k+1} = x^{k + 2} - x^{k + 1}$:


$\begin{align}
x^{k + 2} - x^{k + 1} = (I - D^{-1} A)x^{k+ 1} + D^{-1}b - ((I - D^{-1} A)x^{k} + D^{-1}b)\\
\iff x^{k + 2} - x^{k + 1} = (I - D^{-1} A)(x^{k+ 1} - x^{k})\\
\iff e^{k+2} = (I - D^{-1} A)e^{k + 1} \\
\iff e^{k+2} = Be^{k + 1}\\
\iff e^{k+1} = B^{k + 1}e^{0}
\end{align}
$

#### Theorems

* $\lim_{k\to\infty} \lVert B^k \rVert = 0$ is a necesssary and sufficient condition for the convergence of Jacobi method

* what ever starting point $x^0$ the method converge for the systems where A is strictly diagonally dominant, it means what ever square submatrice  we consider from A, this submatrice is inversible

## 2-Gauss Seidel Method

Gauss seidel method, has nearly the same matricial expression than jacobi method, but it is a sequential
method it can't be parallelized in oposition to jacobi that can be

Before giving the matricial expression we are going to explain the idea and what happens:

The idea is still to construct a sequence $x^n$ that converge to the solution of the system $Ax=b$.

Given a kth point in the sequence $x^k = (x_1^{k}, .... x_n^{k}) \in {\Bbb R}^n$, to compute $x^{k+1} = (x_1^{k + 1}, .... x_n^{k + 1})$, we compute element by element by solving (under the condition $aii \neq 0$) the equations :

for $i \in [1, n]$:

$a_{i1}x_1^{k}+...+a_{ii}x_i^{k+1}+....+a_{in}x_1^{k}=b_i$ we get $x_i^{k+1}$ at equation (i)

after doing that for all $i \in [1, n]$, we call it an iteration.

Now let have the matricial form for those operations:

$A = D - E - F$


$\begin{align}
A x = b\\
\iff& (D - E - F)x = b\\
\iff& (D - E - F)x = b\\
\iff& (D - E)x = Fx + b\\
\end{align}
$

that equation $(D - E)x^{k + 1} = Fx^k + b$  is equivalent to one Gauss seidel iteration

### Condition of existence :

#### Theorems
Gauss Seidel method converge if
* A is defined matrix positive

* A is strictly diagonally dominant

###  1 Jacobi Method

#### 1.1 - Matrix constructions

In [1]:
# We take a symetric definite-positve matrix to fit the rules of algorithm convergences
# and we take a b(1,1,1) we are hoping to find the same thing with jasobi and gauss
import numpy as np
A = np.array([[2,-1,0],[-1,2,-1],[0,-1,2]])
b = np.array([1, 1, 1]).reshape(3,1)
print("Matrix A\n",A)
print("\nVector b\n",b)

Matrix A
 [[ 2 -1  0]
 [-1  2 -1]
 [ 0 -1  2]]

Vector b
 [[1]
 [1]
 [1]]


In [2]:
def get_E(A):
    E = [[-A[i,j] if(i > j) else 0 for j in range(A.shape[1])] for i in range(A.shape[0])]
    return np.array(E)
def get_F(A):
    F = [[-A[i,j] if(i < j) else 0 for j in range(A.shape[1])] for i in range(A.shape[0])]
    return np.array(F)
def get_D(A):
    D = [[A[i,j] if(i == j) else 0 for j in range(A.shape[1])] for i in range(A.shape[0])]
    return np.array(D)

In [3]:
E = get_E(A)
F = get_F(A)
D = get_D(A)
print("F is : \n", F)
print("\nD is : \n", D)
print("\nE is : \n", E)

F is : 
 [[0 1 0]
 [0 0 1]
 [0 0 0]]

D is : 
 [[2 0 0]
 [0 2 0]
 [0 0 2]]

E is : 
 [[0 0 0]
 [1 0 0]
 [0 1 0]]


In [4]:
# our result seems to be confirmed
np.allclose(D - E - F, A)


True

#### 1.2 - Jacobi algorithm

In [5]:
#a random point in a "space" looks like
x0 = np.random.rand(3).reshape(A.shape[0], 1) * 10000
x0

array([[5886.65827897],
       [2702.92548757],
       [2387.194153  ]])

In [6]:
# we construct the function loop where we will iterate the sequence
def jacobi(A,b, tol=1e-5):
    E = get_E(A)
    F = get_F(A)
    D = get_D(A)
    B = np.linalg.inv(D) @ (E + F)
    x0 = np.random.rand(3).reshape(A.shape[0], 1) * 10000
    x1 = B @ x0 + np.linalg.inv(D) @ b
    while(np.linalg.norm(x1 - x0) > tol):
        x0 = x1
        x1 = B @ x0 + np.linalg.inv(D) @ b
    return x1

In [7]:
x_jacobi = jacobi(A, b)
x_jacobi 

array([[1.50001013],
       [2.00001557],
       [1.50001013]])

###  2 Gauss seidel Method
#### 2.1 Gauss seidel implem 

In [8]:
# we construct the function loop where we will iterate the sequence (we are borowing some functions from jacobi part)
def gauss_seidel(A,b, tol=1e-5):
    E = get_E(A)
    F = get_F(A)
    D = get_D(A)
    x0 = np.random.rand(3).reshape(A.shape[0], 1) * 10000
    B = np.linalg.inv(D - E) @ F
    x1 = (D - E) @ x0
    while(np.linalg.norm(x1 - x0) > tol):
        x0 = x1
        x1 = B @ x0 + np.linalg.inv(D - E) @ b
    return x1

In [9]:
x_gauss_seidel = gauss_seidel(A, b)
x_gauss_seidel

array([[1.50000363],
       [2.00000363],
       [1.50000182]])

In [10]:
np.allclose(x_jacobi, x_gauss_seidel)

True

Thank you ! hope that i have brought some value in Gauss seidel and Jacobi method